In [4]:
import random

import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from Extract_Observation_bt import extract_observation_bt
from Extract_Simulated_bt import extract_simulated_bt
from Extract_Observation_Tem import extract_observation_tem
from Extract_Inversion_Tem import extract_inversion_tem

In [5]:
# 加载神经网路模型
model_path = r'I:\Data\Personal_Data\graduation_project\Code\Machine_Learning\model_Tem'
model = tf.keras.models.load_model(model_path)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 7)                 56        
_________________________________________________________________
dense_1 (Dense)              (None, 13)                104       
_________________________________________________________________
dense_2 (Dense)              (None, 47)                658       
Total params: 818
Trainable params: 818
Non-trainable params: 0
_________________________________________________________________


In [6]:
# 加载观测亮温数据 并进入神经网络
years = ['2007']
o_bt = extract_observation_bt(years)
o_bt.drop(o_bt.columns[:14], axis=1, inplace=True)
o_bt.drop(o_bt.columns[7:], axis=1, inplace=True)
o_bt.dropna(inplace=True)

In [7]:
# 模拟亮温数据进入神经网络
s_bt = extract_simulated_bt(years)

# 将模拟亮温的00和12修改为08和20
def trans_time(i):
    if i[-2:] == '00':
        m = '08'
    elif i[-2:] == '12':
        m = '20'
    return i[:-2] + m
s_bt.index = [trans_time(i) for i in s_bt.index]
s_bt.drop(s_bt.columns[:5], axis=1, inplace=True)


In [8]:
# 加载微波辐射计自带神经网路反演得到气象数据
i_tem = extract_inversion_tem(years)
i_tem.dropna(inplace=True)
i_tem.shape

(377, 47)

In [9]:
for i in i_tem.index:
    if(i_tem.loc[i].mean())<0:
        i_tem.drop(i, inplace=True)

In [10]:
# 加载榆中站探测数据
o_tem = extract_observation_tem(years)

# 转换海拔高度格式
o_tem.columns = i_tem.columns
o_tem.index = [trans_time(i) for i in o_tem.index]


In [11]:
print(f'o_bt shape is {o_bt.shape}')
print(f's_bt shape is {s_bt.shape}')
print(f'o_tem shape is {o_tem.shape}')
print(f'i_tem shape is {i_tem.shape}')

o_bt shape is (378, 7)
s_bt shape is (723, 7)
o_tem shape is (723, 47)
i_tem shape is (377, 47)


In [12]:
def norm(x):
    return (x - x.describe().transpose()['mean']) / x.describe().transpose()['std']
o_bt = norm(o_bt)
s_bt = norm(s_bt)

In [17]:
cols = ['s_bt o_tem', 'o_tem i_tem']
cc = pd.DataFrame(columns=cols)

def draw(i):
    plt.figure(figsize=(5, 10))
    Height = o_tem.columns
    X1 = model.predict(pd.DataFrame(s_bt.loc[i]).T).T
    X2 = model.predict(pd.DataFrame(o_bt.loc[i]).T).T
    X3 = pd.to_numeric(i_tem.loc[i])
    # plt.plot(X1, Height, label='s_bt', linestyle=':')
    # plt.plot(X2, Height, label='o_bt', linestyle='-')
    # plt.plot(o_tem.loc[i], Height, label='o_tem', linestyle='dashdot')
    # plt.plot(i_tem.loc[i], Height, label='i_tem', linestyle='dotted')
    # plt.legend()
    # plt.grid()
    # plt.xlabel('Temperature K')
    # plt.ylabel('Altitude km')
    # plt.title(i)
    # path = r'I:\Data\Personal_Data\graduation_project\Graduation_thesis\Atlas\Chapter6\Tem\\'
    # plt.savefig(path+i.replace('/', '').replace(':', '')+'.png',
    #             bbox_inches = 'tight')
    # plt.show()
    # print(f's_bt 和 o_bt 相关系数为{np.corrcoef(X1.T, X2.T)[0, 1].round(4)}')
# print(f's_bt 和 o_tem 相关系数为{np.corrcoef(X1.T, o_tem.loc[i].T)[0, 1].round(4)}')
    # print(f's_bt 和 i_tem 相关系数为{np.corrcoef(X1.T, X3.T)[0, 1].round(4)}')
    # print(f'o_bt 和 o_tem 相关系数为{np.corrcoef(X2.T, o_tem.loc[i].T)[0, 1].round(4)}')
    # print(f'o_bt 和 i_tem 相关系数为{np.corrcoef(X2.T, X3.T)[0, 1].round(4)}')
# print(f'o_tem 和 i_tem 相关系数为{np.corrcoef(o_tem.loc[i].T, X3.T)[0, 1].round(4)}')
    cc.loc[i] = [np.corrcoef(X1.T, o_tem.loc[i].T)[0, 1].round(4),
                 np.corrcoef(o_tem.loc[i].T, X3.T)[0, 1].round(4)]

# i = i_tem.index[random.randrange(0, i_tem.shape[0])]
for i in i_tem.index:
    if (i in s_bt.index)and(i in o_tem.index)and(i in o_bt.index):
        # print(i)
        draw(i)

D:\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """


<Figure size 360x720 with 0 Axes>

<Figure size 360x720 with 0 Axes>

<Figure size 360x720 with 0 Axes>

<Figure size 360x720 with 0 Axes>

...

<Figure size 360x720 with 0 Axes>

<Figure size 360x720 with 0 Axes>

<Figure size 360x720 with 0 Axes>

<Figure size 360x720 with 0 Axes>

<Figure size 360x720 with 0 Axes>

In [28]:
cc.shape
print(cc.mean().round(4))
(cc['s_bt o_tem'] > cc['o_tem i_tem']).sum()

s_bt o_tem     0.9450
o_tem i_tem    0.9365
dtype: float64


289